In [1]:
from paho.mqtt import client as mqtt_client
from datetime import datetime
import random

In [2]:
broker = 'broker.hivemq.com'
port = 1883
client_id = f'n1-sd-hidroponia-{random.randint(0, 1000)}'
topics = {
    'n1_sd_hidro_with_ph_get_temp': 'Solicitado valor de PH com base na temperatura',
    'n1_sd_hidro_ph_get_result': 'Solicitado calculo resultado do PH',
    'n1_sd_hidro_ph_calculate_result': 'Calculado resultado do PH',
    'n1_sd_hidro_ph_return_result': 'Envia resultado do PH',
}

In [3]:
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Conectado ao MQTT Broker!")
    else:
        print("Falha ao se conectar, código %d\n", rc)

In [4]:
client = mqtt_client.Client(client_id)
client.on_connect = on_connect
client.connect(broker, port)
client.loop_start()

Conectado ao MQTT Broker!


In [5]:
def publish(topic, msg):
    result = client.publish(topic, msg)
    status = result[0]
    if status == 0:
        print(f"Enviado `{msg}` para `{topics[topic]}`")
    else:
        print(f"Falha ao enviar mensagem para o tópico {topic}")

In [6]:
def subscribe(topic):
    def on_message(client, userdata, msg):
        now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        print(f"Recebido `{msg.payload.decode()}` para `{topics[msg.topic]}` em `{now}`")
        
        if msg.topic == 'n1_sd_hidro_with_ph_get_temp':
            publish('n1_sd_hidro_ph_get_result', msg.payload.decode())
        if msg.topic == 'n1_sd_hidro_ph_calculate_result':
            publish('n1_sd_hidro_ph_return_result', msg.payload.decode())

    client.subscribe(topic)
    client.on_message = on_message

In [7]:
subscribe('n1_sd_hidro_with_ph_get_temp')
subscribe('n1_sd_hidro_ph_calculate_result')

Recebido `25` para `Solicitado valor de PH com base na temperatura` em `15/10/2022 14:32:00`
Enviado `25` para `Solicitado calculo resultado do PH`
Recebido `12.630693069306933` para `Calculado resultado do PH` em `15/10/2022 14:32:00`
Enviado `12.630693069306933` para `Envia resultado do PH`


In [ ]:
client.loop_stop()